## Pre process
> all thats happening here is the some basic cleaning of notes and rewriting with a pipe delimiter so they can be read into spark in azure

> and the creation of the words of interest which are more predictive

In [1]:
pwd

'/home/ucsddse230/work/final_project/pre_azure'

In [2]:
!pip install nltk


The directory '/home/jovyan/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/jovyan/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
## note that ntlk is only being used on a small file to find disgnosis and procedures
from nltk import word_tokenize
import nltk
import pandas as pd
import string

In [5]:
# if not downloaded
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ucsddse230/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

> link to code downloads
https://www.cms.gov/Medicare/Coding/ICD9ProviderDiagnosticCodes/codes.html

In [6]:
path_to_data='/home/ucsddse230/work/final_project/pre_azure'

# create list of idc9 codes 

In [7]:
def preprocess_text(df):
    # This function preprocesses the text by filling not a number and replacing new lines ('\n') and carriage returns ('\r')
    df.TEXT = df.TEXT.fillna(' ')
    df.TEXT = df.TEXT.str.replace('\n',' ')
    df.TEXT = df.TEXT.str.replace('\r',' ')
    return df



In [8]:
#load idc9 descriptions
notes_df = pd.read_table("{}/idc9_short.txt".format(path_to_data),header=None)
#collapse to string
dirty_idc9=" ".join(list(notes_df[0]))
#turn to pandas
dirty_df=pd.DataFrame([dirty_idc9],columns=['TEXT'])
#remove /n /r and fill na
cleener_idc9=preprocess_text(dirty_df)
#get values
idc9_text=cleener_idc9["TEXT"][0]
#create translate dictionary
punc_list = string.punctuation+'0123456789'
t = str.maketrans(dict.fromkeys(punc_list, " "))
#use translate dictionary
idc9_text = idc9_text.lower().translate(t)
idc9tokens = word_tokenize(idc9_text)

## Create CSV Filter of IDC Procedure and Diagnosis Words

In [9]:
pd.DataFrame(list(set(idc9tokens )), columns=['IDC_filter']).to_csv('IDC9_filter.csv', index=False)

In [10]:
print(list(set(idc9tokens )))

['mature', 'erythematos', 'alexia', 'astringent', 'pg', 'fibros', 'bedclothes', 'effects', 'alveol', 'preop', 'phys', 'sacrum', 'cerebellum', 'multi', 'elev', 'environ', 'dvc', 'altitude', 'dep', 'maxilla', 'glutethimide', 'hypertonicity', 'elsewhere', 'ankylosis', 'persn', 'dura', 'labia', 'fing', 'proliferat', 'replaced', 'bite', 'laryngopharyngitis', 'ath', 'nd', 'glaucomatous', 'sublux', 'articulation', 'pica', 'pilonidal', 'sensory', 'wood', 'hdache', 'ligament', 'vesicant', 'enterocoltis', 'rheumatic', 'insect', 'somatoform', 'eruptions', 'membranoprolif', 'myelophthisis', 'military', 'rick', 'postsurgical', 'pectus', 'maj', 'tympnoscler', 'shortn', 'mcrg', 'occup', 'polishes', 'dendritic', 'hemolyt', 'medulloadrenal', 'mot', 'quills', 'aft', 'trigem', 'sterile', 'prem', 'eyes', 'tor', 'erythem', 'volcanic', 'conduction', 'evid', 'fordyce', 'rflx', 'ulceratve', 'fibrinolysis', 'lftsded', 'soemmering', 'actv', 'epidem', 'paralytic', 'hyperemesis', 'solium', 'succinimides', 'interp

## preprocess notes
> ideally this is done in hive or some other ETL that can write to pipe delimited that spark has no problem reading
> for some reason in current state only pandas can import 

In [12]:
notes_df = pd.read_csv("{}/NOTEEVENTS.csv".format(path_to_data))
##subset to used columns discharge summary
notes_filt_df=notes_df[['SUBJECT_ID','HADM_ID','CATEGORY','TEXT']][notes_df.CATEGORY=='Discharge summary']
#clean
notes_filt_df=preprocess_text(notes_filt_df )


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
notes_filt_df.to_csv('notes_discharge_pd.csv',index=False, sep="|")

## Ready for upload into azure
```python
'notes_discharge_pd.csv' , 'ADMISSIONS.CSV'  and 'IDC9_filter.csv'
```

